### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2025-10-26 01:53:52--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [following]
--2025-10-26 01:53:52--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6c08865a2afc498bdc4afdedd5.dl.dropboxusercontent.com/cd/0/inline/Cz50xIwADoeDT2xppuTrh0F_gZAEF-tC53lhf1JRNtyrklmDPYku389RgWKfGQHsjYUAGhmSsaiBZCM8P8LYPkVGXC27XbLecqHXjlQFS9UQr8O0_GEHU7g-C7dbHRSHwkU/file?dl=1# [following]
--2025-10-26 01:53:52--  https://uc6c08865a2afc498bdc4afdedd5.dl.dropboxuse

,author,day,id,link,month,summary,tag,title,year
20445,"[{'name': 'D. A. Terletskyi'}, {'name': 'A. I....",4,1602.01628v2,"[{'rel': 'related', 'href': 'http://dx.doi.org...",2,This article generalizes object-oriented dynam...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Fuzzy Object-Oriented Dynamic Networks. II,2016
33311,"[{'name': 'J. Pradeep'}, {'name': 'E. Srinivas...",2,1103.0365v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",3,An off-line handwritten alphabetical character...,"[{'term': 'stat.CO', 'scheme': 'http://arxiv.o...",Diagonal Based Feature Extraction for Handwrit...,2011
9615,"[{'name': 'Anne-Laure Boulesteix'}, {'name': '...",13,1208.2651v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",8,In a context where most published articles are...,"[{'term': 'stat.CO', 'scheme': 'http://arxiv.o...",A Plea for Neutral Comparison Studies in Compu...,2012
11377,"[{'name': 'P Satya Jayadev'}, {'name': 'Aravin...",19,1511.06063v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,"Consumers with low demand, like households, ar...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",A Novel Approach for Phase Identification in S...,2015
39214,"[{'name': 'Adriana Gonzalez'}, {'name': 'Véron...",19,1412.6279v3,"[{'rel': 'related', 'href': 'http://dx.doi.org...",12,Context: Characterization of instrumental effe...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Non-parametric PSF estimation from celestial t...,2014


In [ ]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [ ]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer

from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

lines = [' '.join(tokenizer.tokenize(line.lower())) for line in lines]

In [ ]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [ ]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        tokens = line.split()
        # Add EOS token at the end
        tokens.append(EOS)

        for i in range(len(tokens)):
            # Build prefix of length (n-1)
            prefix_start = max(0, i - n + 1)
            prefix = tokens[prefix_start:i]

            # Pad with UNK if prefix is shorter than (n-1)
            prefix = [UNK] * (n - 1 - len(prefix)) + prefix

            # Current token is the next token
            next_token = tokens[i]

            # Update counts
            counts[tuple(prefix)][next_token] += 1

    return counts


In [ ]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [ ]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for prefix, token_counts in counts.items():
          total_count = sum(token_counts.values())
          for token, count in token_counts.items():
              self.probs[prefix][token] = count / total_count

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [ ]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [ ]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [ ]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    possible_tokens = lm.get_possible_next_tokens(prefix)

    if len(possible_tokens) == 0:
        return EOS

    tokens = list(possible_tokens.keys())
    probs = np.array(list(possible_tokens.values()))

    if temperature == 0.0:
        # Always take the most likely token
        return tokens[np.argmax(probs)]
    else:
        # Apply temperature
        probs = probs ** (1.0 / temperature)
        probs = probs / np.sum(probs)  # Normalize

        # Sample from the distribution
        return np.random.choice(tokens, p=probs)

In [ ]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [ ]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial decision making which is a task from the training corpus with article headlines acting as a strategy inspired by the stochastic nature of web services written in probabilistic deductive databases , called proxsvrg +. it recovers structural features in meaning - preserving algorithm for shortest path routing problem referred to as hierarchical softmax or sampling the state of the proposed nmt system on several data mining . however , these ideas can quickly reach a logical approach . the results over three patient ' s terms refer to as stochastic gradient descent in multi - sense reasoning ', which allows


In [ ]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between a pair of data , and a new approach to a set of $ n $ is the problem of learning a joint representation of a number of clusters and a discriminator and generator networks ; the aim of this approach is to be a key role in the presence of noise and outliers . the results of our approach is to present a novel approach to this end , we propose a novel approach , the performance of the proposed algorithm is capable of learning algorithms . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [ ]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    total_log_prob = 0.0
    total_tokens = 0

    for line in lines:
        tokens = line.split()
        tokens.append(EOS)

        for i in range(len(tokens)):
            # Build prefix
            prefix_start = max(0, i - lm.n + 1)
            prefix_tokens = tokens[prefix_start:i]
            prefix = ' '.join(prefix_tokens)

            # Get probability of current token
            next_token = tokens[i]
            prob = lm.get_next_token_prob(prefix, next_token)

            # Compute log probability with minimum threshold
            log_prob = np.log(prob) if prob > 0 else min_logprob
            log_prob = max(log_prob, min_logprob)

            total_log_prob += log_prob
            total_tokens += 1

    # Compute perplexity: exp(-1/N * sum(log_probs))
    perplexity_value = np.exp(-total_log_prob / total_tokens)

    return perplexity_value

In [ ]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [ ]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259043346743296.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [ ]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 1832.66878
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [ ]:
class KneserNeyLanguageModel(NGramLanguageModel):
    """Kneser-Ney smoothed language model"""
    def __init__(self, lines, n, delta=0.75):
        self.n = n
        self.delta = delta
        # Get raw counts
        counts = count_ngrams(lines, n)
        # Build vocabulary
        self.vocab = set()
        for token_counts in counts.values():
            self.vocab.update(token_counts.keys())

        # Store counts for all n-gram levels
        self.counts = {}
        for i in range(1, n + 1):
            self.counts[i] = count_ngrams(lines, i)

        # Compute continuation counts for lower-order models
        self.continuation_counts = {}
        for order in range(1, n):
            self.continuation_counts[order] = defaultdict(int)
            for prefix, token_counts in self.counts[order + 1].items():
                for token in token_counts:
                    # Count how many unique contexts this token appears in
                    shorter_prefix = prefix[-(order-1):] if order > 1 else ()
                    self.continuation_counts[order][(shorter_prefix, token)] += 1

    def _get_lambda(self, prefix_tuple):
        """Compute normalization constant lambda"""
        if prefix_tuple not in self.counts[len(prefix_tuple) + 1]:
            return 0.0

        token_counts = self.counts[len(prefix_tuple) + 1][prefix_tuple]
        num_unique_continuations = len(token_counts)
        total_count = sum(token_counts.values())

        return self.delta * num_unique_continuations / total_count

    def _pkn_unigram(self, token):
        """Compute unigram probability using continuation counts"""
        if self.n == 1:
            # Standard unigram probability
            total = sum(self.counts[1][()].values())
            return self.counts[1][()].get(token, 0) / total if total > 0 else 0.0
        else:
            # Continuation probability for higher-order models
            total_continuations = sum(self.continuation_counts[1].values())
            token_continuations = sum(
                count for (prefix, t), count in self.continuation_counts[1].items()
                if t == token
            )
            return token_continuations / total_continuations if total_continuations > 0 else 0.0

    def _pkn_recursive(self, prefix_tuple, next_token, order):
        """Recursively compute Kneser-Ney probability for given order"""
        # Base case: unigram
        if order == 1:
            return self._pkn_unigram(next_token)

        # Get counts for this context
        token_counts = self.counts[order].get(prefix_tuple, Counter())
        count = token_counts.get(next_token, 0)
        total_count = sum(token_counts.values())

        # First term: discounted probability
        if total_count > 0:
            first_term = max(0, count - self.delta) / total_count
        else:
            first_term = 0.0

        # Second term: backoff with lambda
        lambda_val = self._get_lambda(prefix_tuple)

        # Recursively compute lower-order probability
        shorter_prefix = prefix_tuple[1:] if len(prefix_tuple) > 0 else ()
        lower_prob = self._pkn_recursive(shorter_prefix, next_token, order - 1)

        return first_term + lambda_val * lower_prob

    def get_next_token_prob(self, prefix, next_token):
        """Compute P(next_token | prefix) using Kneser-Ney smoothing"""
        prefix_tokens = prefix.split()
        prefix_tokens = prefix_tokens[max(0, len(prefix_tokens) - self.n + 1):]
        prefix_tokens = [UNK] * (self.n - 1 - len(prefix_tokens)) + prefix_tokens
        prefix_tuple = tuple(prefix_tokens)

        return self._pkn_recursive(prefix_tuple, next_token, self.n)

    def get_possible_next_tokens(self, prefix):
        """Get all possible next tokens with their probabilities"""
        return {token: self.get_next_token_prob(prefix, token)
                for token in self.vocab}

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
# Test different delta values
for n in (1, 2, 3):
    best_delta = None
    best_ppx = float('inf')

    for delta in [0.1, 0.25, 0.5, 0.75, 0.9]:
        lm = KneserNeyLanguageModel(train_lines, n=n, delta=delta)
        ppx = perplexity(lm, test_lines)
        print(f"N = {n}, delta = {delta}, Perplexity = {ppx:.5f}")

        if ppx < best_ppx:
            best_ppx = ppx
            best_delta = delta

    print(f"Best for N={n}: delta={best_delta}, Perplexity={best_ppx:.5f}\n")

N = 1, delta = 0.1, Perplexity = 1832.23136
N = 1, delta = 0.25, Perplexity = 1832.23136
N = 1, delta = 0.5, Perplexity = 1832.23136
N = 1, delta = 0.75, Perplexity = 1832.23136
N = 1, delta = 0.9, Perplexity = 1832.23136
Best for N=1: delta=0.1, Perplexity=1832.23136

